# 如何运行自定义函数

:::info 预备知识

本指南假定您熟悉以下概念：

- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel)
- [链式调用可运行对象](/docs/how_to/sequence/)

:::

您可以将任意函数用作 [Runnables](https://api.js.langchain.com/classes/langchain_core.runnables.Runnable.html)。当您需要格式化数据或需要其他 LangChain 组件未提供的功能时，这非常有用。作为 Runnables 使用的自定义函数被称为 [`RunnableLambdas`](https://api.js.langchain.com/classes/langchain_core.runnables.RunnableLambda.html)。

请注意，这些函数的所有输入都必须是 SINGLE 参数。如果您的函数接受多个参数，您应该编写一个接受单个字典输入并将其解包为多个参数的包装函数。

本指南将涵盖以下内容：

- 如何使用 `RunnableLambda` 构造函数从自定义函数显式创建一个可运行对象
- 在链式调用中使用自定义函数时将其强制转换为可运行对象
- 如何在自定义函数中接收和使用运行元数据
- 如何通过让自定义函数返回生成器来实现流式传输

## 使用构造函数

以下示例中，我们使用 `RunnableLambda` 方法显式包装我们的自定义逻辑：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/openai @langchain/core
</Npm2Yarn>
```

In [1]:
import { StringOutputParser } from "@langchain/core/output_parsers";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableLambda } from "@langchain/core/runnables";
import { ChatOpenAI } from "@langchain/openai";

const lengthFunction = (input: { foo: string }): { length: string } => {
  return {
    length: input.foo.length.toString(),
  };
};

const model = new ChatOpenAI({ model: "gpt-4o" });

const prompt = ChatPromptTemplate.fromTemplate("What is {length} squared?");

const chain = RunnableLambda.from(lengthFunction)
  .pipe(prompt)
  .pipe(model)
  .pipe(new StringOutputParser());

await chain.invoke({ "foo": "bar" });

"3 squared is \\(3^2\\), which means multiplying 3 by itself. \n" +
  "\n" +
  "\\[3^2 = 3 \\times 3 = 9\\]\n" +
  "\n" +
  "So, 3 squared"... 6 more characters

## 链中的自动强制转换

在使用 [`RunnableSequence.from`](https://api.js.langchain.com/classes/langchain_core.runnables.RunnableSequence.html#from) 静态方法将自定义函数用于链中时，可以省略显式的 `RunnableLambda` 创建，而依赖强制转换机制。

以下是一个简单的示例，展示了一个函数如何接收模型的输出并返回其前五个字母：

In [2]:
import { RunnableSequence } from "@langchain/core/runnables";

const storyPrompt = ChatPromptTemplate.fromTemplate("Tell me a short story about {topic}");

const storyModel = new ChatOpenAI({ model: "gpt-4o" });

const chainWithCoercedFunction = RunnableSequence.from([
  storyPrompt,
  storyModel,
  (input) => input.content.slice(0, 5),
]);

await chainWithCoercedFunction.invoke({ "topic": "bears" });

"Once "

请注意，我们不需要将自定义函数 `(input) => input.content.slice(0, 5)` 包装在 `RunnableLambda` 方法中。该自定义函数会被**强制转换**为一个可运行对象。更多信息请参见[此部分](/docs/how_to/sequence/#coercion)。

## 传递运行元数据

Runnable lambda 可以选择性地接受一个 [RunnableConfig](https://api.js.langchain.com/interfaces/langchain_core.runnables.RunnableConfig.html) 参数，通过该参数可以向嵌套运行中传递回调函数、标签以及其他配置信息。

In [3]:
import { type RunnableConfig } from "@langchain/core/runnables";

const echo = (text: string, config: RunnableConfig) => {
  const prompt = ChatPromptTemplate.fromTemplate("Reverse the following text: {text}");
  const model = new ChatOpenAI({ model: "gpt-4o" });
  const chain = prompt.pipe(model).pipe(new StringOutputParser());
  return chain.invoke({ text }, config);
};

const output = await RunnableLambda.from(echo).invoke("foo", {
  tags: ["my-tag"],
  callbacks: [{
    handleLLMEnd: (output) => console.log(output),
  }],
});

{
  generations: [
    [
      {
        text: "oof",
        message: AIMessage {
          lc_serializable: true,
          lc_kwargs: [Object],
          lc_namespace: [Array],
          content: "oof",
          name: undefined,
          additional_kwargs: [Object],
          response_metadata: [Object],
          tool_calls: [],
          invalid_tool_calls: []
        },
        generationInfo: { finish_reason: "stop" }
      }
    ]
  ],
  llmOutput: {
    tokenUsage: { completionTokens: 2, promptTokens: 13, totalTokens: 15 }
  }
}


# 流式传输

你可以在链中使用生成器函数（即使用 `yield` 关键字并表现为迭代器的函数）。

这些生成器的签名应为 `AsyncGenerator<Input> -> AsyncGenerator<Output>`。

它们适用于以下场景：
- 实现自定义输出解析器
- 在保留流式功能的同时修改前一步骤的输出

下面是一个针对逗号分隔列表的自定义输出解析器示例。首先，我们创建一个链来生成这样的列表文本：

In [4]:
const streamingPrompt = ChatPromptTemplate.fromTemplate(
  "Write a comma-separated list of 5 animals similar to: {animal}. Do not include numbers"
);

const strChain = streamingPrompt.pipe(model).pipe(new StringOutputParser());

const stream = await strChain.stream({ animal: "bear" });

for await (const chunk of stream) {
  console.log(chunk);
}


Lion
,
 wolf
,
 tiger
,
 cougar
,
 leopard



接下来，我们定义一个自定义函数，该函数将聚合当前流式输出，并在模型生成列表中的下一个逗号时将其返回：

In [5]:
// This is a custom parser that splits an iterator of llm tokens
// into a list of strings separated by commas
async function* splitIntoList(input) {
  // hold partial input until we get a comma
  let buffer = "";
  for await (const chunk of input) {
    // add current chunk to buffer
    buffer += chunk;
    // while there are commas in the buffer
    while (buffer.includes(",")) {
      // split buffer on comma
      const commaIndex = buffer.indexOf(",");
      // yield everything before the comma
      yield [buffer.slice(0, commaIndex).trim()];
      // save the rest for the next iteration
      buffer = buffer.slice(commaIndex + 1);
    }
  }
  // yield the last chunk
  yield [buffer.trim()];
}

const listChain = strChain.pipe(splitIntoList);

const listChainStream = await listChain.stream({"animal": "bear"});

for await (const chunk of listChainStream) {
  console.log(chunk);
}

[ "wolf" ]
[ "lion" ]
[ "tiger" ]
[ "cougar" ]
[ "cheetah" ]


调用它会得到一个完整的值数组：

In [7]:
await listChain.invoke({"animal": "bear"})

[ "lion", "tiger", "wolf", "cougar", "jaguar" ]

## 下一步
现在你已经学习了几种在链中使用自定义逻辑的方法，以及如何实现流式传输。

要了解更多信息，请参阅本节中有关可运行对象的其他操作指南。